# Reset Lakehouse

Removes all Silver and Gold tables and databases. Use this to start fresh.

**WARNING:** This will permanently delete all data in the Silver (ag) and Gold (au) databases.

In [ ]:
import os

def get_env(var_name, default=None):
    return os.environ.get(var_name, default)

SILVER_DB = get_env("SILVER_DB", default="ag")
GOLD_DB = get_env("GOLD_DB", default="au")

print(f"This will reset: SILVER_DB={SILVER_DB}, GOLD_DB={GOLD_DB}")
print("\nWARNING: All tables will be permanently deleted!")

In [ ]:
def drop_all_tables(database):
    """Drop all tables in a database."""
    try:
        tables = spark.sql(f"SHOW TABLES IN {database}").collect()
        print(f"\nDropping {len(tables)} tables from {database}:")
        for table in tables:
            table_name = table.tableName
            try:
                spark.sql(f"DROP TABLE IF EXISTS {database}.{table_name}")
                print(f"  Dropped: {database}.{table_name}")
            except Exception as e:
                print(f"  Error dropping {table_name}: {e}")
    except Exception as e:
        print(f"Database {database} does not exist or error: {e}")

def drop_database(database):
    """Drop a database after dropping all its tables."""
    try:
        spark.sql(f"DROP DATABASE IF EXISTS {database} CASCADE")
        print(f"Dropped database: {database}")
    except Exception as e:
        print(f"Error dropping database {database}: {e}")

In [ ]:
print("="*60)
print("RESETTING GOLD DATABASE")
print("="*60)

drop_all_tables(GOLD_DB)
drop_database(GOLD_DB)

In [ ]:
print("="*60)
print("RESETTING SILVER DATABASE")
print("="*60)

drop_all_tables(SILVER_DB)
drop_database(SILVER_DB)

In [ ]:
print("\n" + "="*60)
print("RESET COMPLETE")
print("="*60)

# Verify databases are gone
databases = [row.databaseName for row in spark.sql("SHOW DATABASES").collect()]
print(f"\nRemaining databases: {databases}")

if SILVER_DB not in databases and GOLD_DB not in databases:
    print(f"\nSuccess: {SILVER_DB} and {GOLD_DB} have been removed.")
else:
    remaining = [db for db in [SILVER_DB, GOLD_DB] if db in databases]
    print(f"\nWarning: These databases still exist: {remaining}")

print("\nTo reload data, run: 02-historical-data-load.ipynb")